In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from time import sleep
from processor.processor import Processor as p
from extractor.tiingo_extractor import TiingoExtractor
from extractor.fred_extractor import FREDExtractor

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")

In [11]:
analysis = []
market.connect()
periods = ["year","quarter","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test = p.column_date_processing(test)
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["rolling_100"] = test["adjclose"].rolling(window=100).mean()
        ep = test.tail(1)["adjclose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjclose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjclose"].item())/test.tail(2).iloc[0]["adjclose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:32<00:00, 15.05it/s]


In [12]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
399,0.040728,0.078431,0.078431,0.086116,SLB,Schlumberger,Energy,Oil & Gas Equipment & Services
214,-0.051059,0.057984,0.057984,0.077855,HAL,Halliburton,Energy,Oil & Gas Equipment & Services
146,-0.483688,0.075332,0.075332,0.069016,DISH,Dish Network,Communication Services,Cable & Satellite
316,-0.130110,0.028260,0.028260,0.053612,MOS,The Mosaic Company,Materials,Fertilizers & Agricultural Chemicals
282,-0.592179,-0.026667,-0.026667,0.047847,LUMN,Lumen Technologies,Communication Services,Alternative Carriers
56,0.158627,0.047785,0.047785,0.047785,BKR,Baker Hughes,Energy,Oil & Gas Equipment & Services
354,-0.155108,0.018953,0.018953,0.047438,PENN,Penn National Gaming,Consumer Discretionary,Casinos & Gaming
17,0.109383,0.036029,0.036029,0.043723,ALB,Albemarle Corporation,Materials,Specialty Chemicals
285,-0.079306,0.022609,0.022609,0.043478,MRO,Marathon Oil,Energy,Oil & Gas Exploration & Production
488,-0.397282,0.029851,0.029851,0.043197,ZION,Zions Bancorporation,Financials,Regional Banks


In [13]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain
GICS Sector,,,,
Energy,-0.019712,0.006846,0.006846,0.031026
Communication Services,0.131269,0.003362,-0.006205,0.003096
Utilities,-0.039976,-0.004965,-0.004965,-0.006724
Real Estate,0.022632,-0.005218,-0.005218,-0.001794
Financials,-0.052378,-0.007571,-0.007571,0.008490
Industrials,0.141077,-0.011210,-0.011210,0.004126
Materials,0.018541,-0.012790,-0.012790,0.015355
Information Technology,0.199162,-0.014820,-0.014820,0.000289
Health Care,0.006660,-0.015394,-0.015032,-0.006136
